In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.100d.txt
/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.50d.txt
/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.200d.txt
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip


In [20]:
train = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
test_labels = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip')

In [21]:
word_to_vec_map = {}
words = set()
with open('../input/glove-global-vectors-for-word-representation/glove.6B.50d.txt') as file:
    for line in file:
        values = line.strip().split()
        curr_word = values[0]
        words.add(curr_word)
        word_to_vec_map[curr_word] = np.array(values[1:], dtype = np.float64)
len(word_to_vec_map)

400000

In [22]:
train.shape, test.shape, test_labels.shape

((159571, 8), (153164, 2), (153164, 7))

In [23]:
train.head(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


Here, the output variables are not exclusive. More than one can occur at same time. 

In [24]:
test.head(20)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
5,0001ea8717f6de06,Thank you for understanding. I think very high...
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...
7,000247e83dcc1211,:Dear god this site is horrible.
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ..."
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...


In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [27]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153164 entries, 0 to 153163
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            153164 non-null  object
 1   comment_text  153164 non-null  object
dtypes: object(2)
memory usage: 2.3+ MB


No null values

In [28]:
train_sentences = train['comment_text'].values
test_sentences = test['comment_text'].values

In [29]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 10000)
from keras.preprocessing.sequence import pad_sequences

In [30]:
max_seq_length = 1000
tokenizer.fit_on_texts(train_sentences)
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
padded_seq_train = pad_sequences(train_sequences, maxlen = max_seq_length)
padded_seq_test = pad_sequences(test_sequences, maxlen = max_seq_length)

In [31]:
index = tokenizer.word_index
len(index)

210337

In [32]:
embedding_matrix = np.zeros((len(index) + 1, 50))
for word, i in index.items():
    temp = word_to_vec_map.get(word)
    if temp is not None:
        embedding_matrix[i] = temp

In [33]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']
y = train[classes].values

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
xTrain, xTest, yTrain, yTest = train_test_split(padded_seq_train, y, test_size = 0.3, random_state = 21)

In [36]:
yTrain.shape

(111699, 6)

In [37]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, Dense, LSTM, GlobalMaxPooling1D, Dropout

In [38]:
embed_layer = Embedding(len(index) + 1, 50, input_length = max_seq_length, weights = [embedding_matrix] )

In [39]:
model = Sequential()
model.add(embed_layer)
model.add(Bidirectional(LSTM(50, return_sequences = True, dropout = 0.1, recurrent_dropout = 0.1)))
model.add(GlobalMaxPooling1D())
model.add(Dense(50, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation = 'sigmoid'))

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 50)          10516900  
_________________________________________________________________
bidirectional (Bidirectional (None, 1000, 100)         40400     
_________________________________________________________________
global_max_pooling1d (Global (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                5050      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 306       
Total params: 10,562,656
Trainable params: 10,562,656
Non-trainable params: 0
____________________________________________

In [41]:
model.compile(loss='binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

In [42]:
history = model.fit(xTrain, yTrain, epochs = 2, batch_size = 128, validation_split = 0.1)

Epoch 1/2
786/786 [==============================] - 1674s 2s/step - loss: 0.0841 - accuracy: 0.8005 - val_loss: 0.0525 - val_accuracy: 0.9944
Epoch 2/2
786/786 [==============================] - 1658s 2s/step - loss: 0.0515 - accuracy: 0.9665 - val_loss: 0.0491 - val_accuracy: 0.9944


In [43]:
result = model.evaluate(xTest,yTest)

1496/1496 [==============================] - 214s 143ms/step - loss: 0.0491 - accuracy: 0.9940


In [44]:
pred = model.predict(padded_seq_test)

In [28]:
pred.shape

(153164, 6)

In [45]:
sample = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')

In [46]:
sample.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5


In [47]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [48]:
sample[classes] = pred

In [49]:
sample.head(20)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.974439,3.029092e-01,0.869210,0.052715,0.789627,0.182595
1,0000247867823ef7,0.006777,4.685082e-05,0.002099,0.000089,0.001869,0.000389
2,00013b17ad220c46,0.004888,5.833626e-05,0.001230,0.000126,0.001297,0.000316
3,00017563c3f7919a,0.001554,5.160010e-06,0.000237,0.000011,0.000313,0.000033
4,00017695ad8997eb,0.006145,4.734318e-05,0.001182,0.000105,0.001420,0.000260
5,0001ea8717f6de06,0.000753,3.085329e-06,0.000141,0.000008,0.000177,0.000025
6,00024115d4cbde0f,0.003124,9.445339e-06,0.000500,0.000028,0.000668,0.000081
7,000247e83dcc1211,0.534823,6.291837e-03,0.122135,0.017006,0.160985,0.027405
8,00025358d4737918,0.083875,2.847612e-04,0.014208,0.000781,0.016015,0.002077
9,00026d1092fe71cc,0.002606,8.001315e-06,0.000483,0.000023,0.000559,0.000059


In [50]:
sample.to_csv('submission.csv', index = False)